<a href="https://colab.research.google.com/github/samko5sam/programming-language-class/blob/main/0320_DataVisualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [4]:
import pandas as pd
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1IkFP9_-C0sdyZNgp3gjeoqPyv3MV2fQqUFKgTKRLF2w/edit?usp=sharing')
worksheet = gsheets.get_worksheet(0)

In [14]:
# 從 Google Sheets 讀取資料

data = worksheet.get_all_values()
df = pd.DataFrame(data)
df.columns=df.iloc[0]
df = df.iloc[1:]
# 將數值欄位轉換為數字類型
numeric_columns = ['除權息年度', '股利', '除息前股價', '填息天數', '年股利', '年殖利率(%)']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')  # 轉為數值，無效數據變為 NaN
df

,除權息年度,股利,除息日,發放日,除息前股價,填息天數,年股利,年殖利率(%)
1,2025,2.70,2025/1/17,2025/2/20,198.05,4,2.70,1.36
2,2024,1.00,2024/7/16,2024/8/9,196.70,95,4.00,2.44
3,2024,3.00,2024/1/17,2024/2/21,131.65,3,4.00,2.44
4,2023,1.90,2023/7/18,2023/8/11,132.00,127,4.50,3.60
5,2023,2.60,2023/1/30,2023/3/7,118.10,2,4.50,3.60
6,2022,1.80,2022/7/18,2022/8/19,115.50,4,5.00,3.78
7,2022,3.20,2022/1/21,2022/3/4,149.10,4,5.00,3.78
8,2021,0.35,2021/7/21,2021/8/24,137.20,2,3.40,2.43
9,2021,3.05,2021/1/22,2021/3/9,143.00,27,3.40,2.43
10,2020,0.70,2020/7/21,2020/8/24,97.05,2,3.60,3.81


In [18]:
import plotly.express as px

# 按年度分組並取平均值
df_grouped = df.groupby('除權息年度')[['年股利', '年殖利率(%)']].mean().reset_index()

# 創建折線圖
fig = px.line(df_grouped,
              x='除權息年度',
              y=['年股利', '年殖利率(%)'],
              title='年度股利與殖利率變化',
              labels={'value': '金額/百分比', '除權息年度': '年度', 'variable': '指標'},
              height=600,
              width=800)

# 更新折線圖，添加數據點標記
fig.update_traces(mode='lines+markers', marker=dict(size=8))  # 添加點並設置大小

# 更新佈局
fig.update_layout(
    yaxis_title="金額(元) / 百分比(%)",
    legend_title="指標",
    template="plotly_white"
)

# 顯示圖表
fig.show()